# Characater to Character Translation

Data Source: http://www.manythings.org/anki/


In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

BATCH_SIZE= 64 # batch size for training
EPOCHS = 100 # number of epochs to train for
LATENT_DIM = 256 # feature dimensions for embedding
NUM_SAMPLES = 10000 # number of samples to train


data_path = '/content/drive/Othercomputers/My Laptop/syr_ads_deep_learning/labs/datasets/fra-eng/fra.txt'

# sample data
# English sentence French Sentence 
# Go.	Va !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)
# Go.	Marche.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)
# Go.	En route !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)
# Go.	Bouge !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)
# Hi.	Salut !	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)
# Hi.	Salut.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)

Loading the data

In [ ]:
# we are reading each character from english and french sentences. Unique characters are stored in the set
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as file_handler:
  lines = file_handler.read().split('\n')
  for line in lines[: min(NUM_SAMPLES, len(lines)-1)]:
    input_text , target_text, _ = line.split('\t')
    input_texts.append(input_text)
    target_text = '\t' + target_text + '\n' # we are using "tab" as the "start sequence" character for the targets and "\n" as "end sequence" character
    target_texts.append(target_text)

    # now creating unique character set - vocaubulary. In this case it will be alphabets of english and french language
    for char in input_text:
      if char not in input_characters:
        input_characters.add(char)

    for char in target_text:
      if char not in target_characters:
        target_characters.add(char)

In [ ]:
input_chars = sorted(list(input_characters))
target_chars = sorted(list(target_characters))
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max([len(text) for text in input_texts ])
max_decoder_seq_length = max([len(text) for text in target_texts ])

In [ ]:
print(f"Number of Samples:{len(input_texts)}")
print(f"Number of Unique Input Tokens (English Characters):{num_encoder_tokens}")
print(f"Number of Unique Target Tokens (French Characters):{num_decoder_tokens}")
print(f"Max sequence length for inputs: {max_encoder_seq_length}") # maximum english sentence length
print(f"Max sequence length for outputs: {max_decoder_seq_length}") # maximum french sentence length

Number of Samples:10000
Number of Unique Input Tokens (English Characters):71
Number of Unique Target Tokens (French Characters):93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [ ]:
# adding index to each character in the sorted list
input_token_index = dict([(char, i) for i, char in enumerate(input_chars)])
output_token_index = dict([(char, i) for i, char in enumerate(target_chars)])

In [ ]:
# intializing one hot encode vectors

# encoder_input_data is a 3D array of shape (num_senteces, max_english_sentence_length, num_english_characters) containing a one-hot vectorization of the English sentences.
encoder_input_data = np.zeros(shape=(len(input_texts), max_encoder_seq_length,  num_encoder_tokens), dtype=np.float32)

# decoder_input_data is a 3D array of shape (num_sentences, max_french_sentence_length, num_french_characters) containg a one-hot vectorization of the French sentences.
decoder_input_data = np.zeros(shape=(len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype=np.float32)

# decoder_target_data is the same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :].
decoder_target_data = np.zeros(shape=(len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype=np.float32)

One Hot Encoding

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.0
  encoder_input_data[i, t+1:, input_token_index[' ']] = 1.0 # padding with spaces
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, output_token_index[char]] = 1.0
    if t>0:
      # decoder_target_data is ahead of decoder_input_data by one timestamp
      # and will not include the start character
      decoder_target_data[i, t-1, output_token_index[char]] = 1.0
  decoder_input_data[i, t+1:, output_token_index[' '] ] = 1.0 # padding with spaces
  decoder_target_data[i, t+1:, output_token_index[' '] ] = 1.0 # padding with spaces

In [ ]:
encoder_input_data.shape

(10000, 15, 71)

LSTM Layers

Encoder

In [ ]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(LATENT_DIM, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs) # we discard encoder_outputs and keep only the states
encoder_states = [state_h, state_c]

Decoder

In [ ]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True)
decoder_outputs, _ , _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

Building the Model

In [ ]:
# define the model that will turn encoder_input_data & decoder_input_data into decoder_target_data

lang_translation_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
lang_translation_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
lang_translation_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, 
                           batch_size=BATCH_SIZE, 
                           epochs = EPOCHS,
                           validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 52s 385ms/step - loss: 1.2014 - accuracy: 0.7155 - val_loss: 1.0609 - val_accuracy: 0.6927
Epoch 2/100
125/125 [==============================] - 48s 385ms/step - loss: 0.9230 - accuracy: 0.7340 - val_loss: 0.9790 - val_accuracy: 0.6999
Epoch 3/100
125/125 [==============================] - 47s 377ms/step - loss: 0.8650 - accuracy: 0.7484 - val_loss: 0.9221 - val_accuracy: 0.7256
Epoch 4/100
125/125 [==============================] - 49s 390ms/step - loss: 0.8169 - accuracy: 0.7600 - val_loss: 0.8666 - val_accuracy: 0.7409
Epoch 5/100
125/125 [==============================] - 46s 369ms/step - loss: 0.7432 - accuracy: 0.7777 - val_loss: 0.8006 - val_accuracy: 0.7494
Epoch 6/100
125/125 [==============================] - 46s 370ms/step - loss: 0.6963 - accuracy: 0.7871 - val_loss: 0.7564 - val_accuracy: 0.7628
Epoch 7/100
125/125 [==============================] - 49s 391ms/step - loss: 0.6857 - accuracy: 0.7888 - val_loss: 1.0401 -

In [ ]:
# save the model
lang_translation_model.save('e2f_model')

Run Inference

In [ ]:
import keras

# Restoring the model and constructing the encoder and decoder for prediction
e2f_model = keras.models.load_model('e2f_model') 
encoder_inputs = e2f_model.inputs[0] # encoder inputs ; input_1
encoder_outputs, enc_state_hidden, enc_state_cell = e2f_model.layers[2].output # encoder lstm ; lstm_1
encoder_states = [enc_state_hidden, enc_state_cell]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = e2f_model.input[1] # decoder input; input_2
dec_states_hidden = keras.Input(shape=(LATENT_DIM,))
dec_states_cell = keras.Input(shape=(LATENT_DIM,))
decoder_input_states = [dec_states_hidden, dec_states_cell]
decoder_lstm = e2f_model.layers[3] 

decoder_output, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_input_states)

# Last Dense Layer
decoder_states = [state_h_dec, state_c_dec]

decoder_dense = e2f_model.layers[4]
decoder_outputs = decoder_dense(decoder_output)

decoder_model = keras.Model([decoder_inputs] + decoder_input_states, [decoder_outputs]+decoder_states)

In [ ]:
e2f_model.layers

Generate Decoded Sentences

In [ ]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in output_token_index.items())

def decode_sentence(input_seq):
  # Encode the input as state vectors
  states_value = encoder_model.predict(input_seq)

  # Generate empty target sequence of length 1 - just initializing for '\t' first character in one sentence
  target_seq = np.zeros((1,1, num_decoder_tokens))
  # Populate the first character of target sequence with the start character.
  target_seq[0,0, output_token_index['\t']] = 1.0

  stop_condition = False
  decoded_sentence = ""
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
    predicted_token_index = np.argmax(output_tokens[0, -1, :])
    predicted_char = reverse_target_char_index[predicted_token_index]
    decoded_sentence += predicted_char
    
    # Exit condition: either hit max length or find stop character.
    if predicted_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
      stop_condition = True
    
    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, predicted_token_index] = 1.0

    # Update states
    states_value = [h, c]
  return decoded_sentence



  

In [ ]:
# decoding first 20 sentences
for sentence_index in range(20):
    # Take one sentence (part of the training set) and decoding it
    input_seq = encoder_input_data[sentence_index:sentence_index+1]
    decoded_sentence = decode_sentence(input_seq)
    print('---------------------------------------------------------------------------------')
    print(f"English: {input_texts[sentence_index]}")
    print(f"French: {target_texts[sentence_index]}")
    print(f"Predicted French Translation: {decoded_sentence}")

1/1 [==============================] - 0s 23ms/step
---------------------------------------------------------------------------------
English: Go.
French: 	Va !

Predicted French Translation: Va !

1/1 [==============================] - 0s 24ms/step
---------------------------------------------------------------------------------
English: Go.
French: 	Marche.

Predicted French Translation: Va !

1/1 [==============================] - 0s 24ms/step
---------------------------------------------------------------------------------
English: Go.
French: 	En route !

Predicted French Translation: Va !

1/1 [==============================] - 0s 25ms/step
---------------------------------------------------------------------------------
English: Go.
French: 	Bouge !

Predicted French Translation: Va !

1/1 [==============================] - 0s 30ms/step
---------------------------------------------------------------------------------
English: Hi.
French: 	Salut !

Predicted French Translation: S